# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [13]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info(show_token=True)


Timelink version: 1.1.9
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: 8dllru14Ylj7xRe8pwhxKkZJKfejPbCZ
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: priceless_shannon
Kleio version requested: latest
Kleio server version: 12.5.570 (2024-02-29 07:39:03)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,aregisters,1
2,attributes,25533
3,class_attributes,70
4,classes,14
5,entities,31691
6,geoentities,359
7,goods,0
8,kleiofiles,31
9,objects,0


### Show the kleio files available 

In [3]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-0-abrev.cli,dehergne-0-abrev.cli,2024-05-16 06:22:15.479836+00:00,V,2024-05-16 06:22:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-16 06:22:35.588672,/rest/reports/sources/dehergne-0-abrev.rpt,/rest/exports/sources/dehergne-0-abrev.xml
1,sources/dehergne-a.cli,dehergne-a.cli,2024-05-15 01:06:05.929768+00:00,V,2024-05-15 01:06:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-15 01:06:24.666007,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
2,sources/dehergne-b.cli,dehergne-b.cli,2024-05-06 14:54:16.003983+00:00,V,2024-05-06 14:54:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-06 15:00:49.294836,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
3,sources/dehergne-c.cli,dehergne-c.cli,2024-05-16 06:22:24.567501+00:00,V,2024-05-16 06:22:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-16 06:22:46.707242,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
4,sources/dehergne-d.cli,dehergne-d.cli,2024-05-16 06:22:20.512796+00:00,V,2024-05-16 06:22:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-16 06:22:51.633689,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
5,sources/dehergne-e.cli,dehergne-e.cli,2024-05-07 13:23:37.217587+00:00,V,2024-05-07 13:23:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-07 13:24:00.323489,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml
6,sources/dehergne-f.cli,dehergne-f.cli,2024-05-16 14:14:48.910709+00:00,V,2024-05-16 14:14:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-16 14:15:02.303668,/rest/reports/sources/dehergne-f.rpt,/rest/exports/sources/dehergne-f.xml
7,sources/dehergne-g.cli,dehergne-g.cli,2024-05-09 11:14:59.805561+00:00,V,2024-05-09 11:14:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-15 01:06:38.353702,/rest/reports/sources/dehergne-g.rpt,/rest/exports/sources/dehergne-g.xml
8,sources/dehergne-h.cli,dehergne-h.cli,2024-04-25 06:20:59.180810+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-25 06:27:13.144595,/rest/reports/sources/dehergne-h.rpt,/rest/exports/sources/dehergne-h.xml
9,sources/dehergne-i.cli,dehergne-i.cli,2024-04-25 06:20:59.183135+00:00,V,2024-04-24 07:10:00+00:00,0,0,I,0,0,No errors,No warnings,2024-04-25 06:27:14.548095,/rest/reports/sources/dehergne-i.rpt,/rest/exports/sources/dehergne-i.xml


## Optionally clean translations 

In [9]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['kleio/dehergne-locations-1701.cli',
 'kleio/dehergne-locations-1644.cli',
 'kleio/dehergne-a.cli',
 'kleio/b1685.cli',
 'kleio/auc-alunos.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [33]:
import logging
logging.basicConfig(level=logging.INFO)

tlnb.update_from_sources()
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

INFO:root:Request translation of T sources/dehergne-p.cli
INFO:root:Importing sources/dehergne-p.cli
INFO:root:Importing sources/dehergne-p.cli
INFO:root:Storing 31 postponed relations


## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [30]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()
imported_files_df[["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                31 non-null     object             
 1   name                31 non-null     object             
 2   size                31 non-null     int64              
 3   directory           31 non-null     object             
 4   modified            31 non-null     datetime64[ns, UTC]
 5   modified_iso        31 non-null     datetime64[ns, UTC]
 6   modified_string     31 non-null     object             
 7   qtime               31 non-null     datetime64[ns, UTC]
 8   qtime_string        31 non-null     object             
 9   source_url          31 non-null     object             
 10  status              31 non-null     object             
 11  translated          31 non-null     datetime64[ns, UTC]
 12  translated_string   31 non-null     ob

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path
0,I,0,0,dehergne-0-abrev.cli,2024-05-16 06:22:35.588672,0,0,sources/dehergne-0-abrev.cli
1,I,0,0,dehergne-a.cli,2024-05-19 05:38:24.324317,0,0,sources/dehergne-a.cli
2,I,0,0,dehergne-b.cli,2024-05-18 03:58:30.652811,0,0,sources/dehergne-b.cli
3,I,0,0,dehergne-c.cli,2024-05-19 06:04:32.533819,0,0,sources/dehergne-c.cli
4,I,0,0,dehergne-d.cli,2024-05-16 06:22:51.633689,0,0,sources/dehergne-d.cli
5,I,0,0,dehergne-e.cli,2024-05-07 13:24:00.323489,0,0,sources/dehergne-e.cli
6,I,0,0,dehergne-f.cli,2024-05-20 09:52:22.806094,0,0,sources/dehergne-f.cli
7,I,0,0,dehergne-g.cli,2024-05-18 04:00:53.383878,0,0,sources/dehergne-g.cli
8,I,0,0,dehergne-h.cli,2024-05-19 05:07:02.264655,0,0,sources/dehergne-h.cli
9,I,0,0,dehergne-i.cli,2024-04-25 06:27:14.548095,0,0,sources/dehergne-i.cli


### Check a translation report

If needed to check for errors.


In [16]:
file_number=2
rpt = tlnb.get_translation_report(imported_files_df, file_number)
print(rpt)


KleioTranslator - server version 12.5 - build 570 2024-02-29 07:39:02
6-5-2024 14-54

Processing data file dehergne-b.cli
-------------------------------------------
Generic Act translation module with geoentities (XML).
     Joaquim Ramos de Carvalho (joaquim@uc.pt) 
** New document: kleio
kleio translation started
Structure: gacto2.str
Prefix: 
Autorel: 
Translation count: 189
Obs: 
** Processing source fonte$dehergne-b
7: lista$dehergne-notices-b
*** End of File

Line 359 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-antonio-de-abreu) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 839 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-alessandro-cicero) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.
Line 1459 "SAME AS" TO EXTERNAL REFERENCE EXPORTED (deh-louis-porquet) CHECK IF IT EXISTS BEFORE IMPORTING THIS FILE.

Structure file: /kleio-home/structures/sources.str
Structure processing report: /kleio-home/structures/sources.srpt
Structure in JSON: /kleio-home/structures/sources.str

### Get the import report for a file

In case there are errors in the import phase check the import report.

In [ ]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

No errors
No warnings
